# Burlington TPOT

In [5]:
from btvFucntions import parse_btv_props
#for tablular data managment 
import pandas as pd 
#for scientific plotting 
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline


### TPOT 

TPOT is a genetic algorithm that creates optimized sklearn pipelines auotmatically.  I have already identified a pipeline that works quite well to predict housing prices but I'd like to see how this compares to what TPOT finds. 

In addititon, TPOT might discover some additional preprocessing steps that may help to improve prediction. The package itself has many different modules that are not documented in the API for building models. It is pretty clear typically how to use them though based on the code generated by TPOT

In [4]:
# look to btvFunctions.py for code used to parse in the data and create the model matrix
# This function is contained in a seperate .py file to keep it from taking up too much space in the notebook
# This also helps with code organization
df = parse_btv_props()


df['SaleDate'] = pd.to_datetime(df['SaleDate'])
modeldf = df.drop(['AccountNumber', 'ParcelID', 'SpanNumber', 'StreetNumber',
       'AlternateNumber', 'Unit', 'CuO1LastName',
       'CuO1FirstName', 'CuO2LastName', 'CuO2FirstName', 'CuO3LastName',
       'CuO3FirstName','LegalReference', 'GrantorLastName', 'FID', 'Baths'], axis = 1)
modeldf['Latitude'] = pd.to_numeric(modeldf['Latitude'])
modeldf['Longitude'] = pd.to_numeric(modeldf['Longitude'])
#modeldf['Sale_Year'] = modeldf['SaleDate'].apply(lambda row: row.year)
modeldf.drop('SaleDate', axis=1, inplace= True)
modeldf = modeldf[(modeldf['LandUse'] == "Single Family") | (modeldf['LandUse'] == "Residential Condo")]
modeldf = modeldf[modeldf['SalePrice']> 0]
modeldf.dropna(inplace = True)
#pulled cityhall gps coords from this https://www.maps.ie/coordinates.html
from geopy.distance import great_circle
distances = []
for i, j in zip(modeldf['Latitude'], modeldf['Longitude']):
    val = (i, j)
    Cityhall = (44.47647568031712, -73.21353835752235)
    dist = great_circle(val,Cityhall).miles
    distances.append(dist)
    
    
    
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor

from sklearn.model_selection import train_test_split



modeldf = pd.get_dummies(modeldf)
target_name = 'SalePrice'
y=modeldf[target_name]
Xframe = modeldf.drop(['SalePrice'], axis=1)
data_names = Xframe.columns
X = Xframe.copy()
# create train-test splits
X_train, X_test, y_train, y_test = train_test_split(X, y)



X_train, X_test, y_train, y_test = train_test_split(X, y)
imp = SimpleImputer(strategy = 'median')
X_train = imp.fit_transform(X_train)
X_test = imp.transform(X_test)
tpot = TPOTRegressor(generations=20, population_size=20, verbosity=2, use_dask=True, n_jobs= -1)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('btv_model.py')

Dataframe dimensions: (4605, 45)


Generation 1 - Current best internal CV score: -2763194709.8879194
Generation 2 - Current best internal CV score: -2755283429.436372
Generation 3 - Current best internal CV score: -2755283429.436372
Generation 4 - Current best internal CV score: -2753376551.2473683
Generation 5 - Current best internal CV score: -2680936547.471691
Generation 6 - Current best internal CV score: -2632685431.529882
Generation 7 - Current best internal CV score: -2632685431.529882
Generation 8 - Current best internal CV score: -2632685431.529882


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: ElasticNetCV(RidgeCV(GradientBoostingRegressor(input_matrix, alpha=0.8, learning_rate=0.001, loss=huber, max_depth=7, max_features=0.8500000000000001, min_samples_leaf=16, min_samples_split=16, n_estimators=100, subsample=0.8)), l1_ratio=0.9500000000000001, tol=0.001)
-6574731662.282968


True